## Proces wyodrębnienia przyzwoitych pseudo-masek

#### Importowanie bibliotek

In [1]:
from pathlib import Path
import shutil
import csv

#### Funkcja `load_ids`
Wczytanie listy identyfikatorów do przeprowadzenia kopiowania

In [2]:
def load_ids(csv_path, id_field = "image_id"):
    ids = set()
    with csv_path.open(newline="", encoding="utf-8") as f:
        reader = csv.DictReader(f)
        for row in reader:
            ids.add(Path(row[id_field]).stem)
    return ids

#### Funkcja `build_file_index`
Funkcja tworzy słownik indeksujący wszystkie pliki w katalogu w którym kluczem jest nazwa pliku bez rozszerzenia, a wartością pełna ścieżka.

In [3]:
def build_file_index(src_dir):
    index = {}
    for p in src_dir.iterdir():
        if p.is_file():
            index[p.stem] = p
    return index

#### Funkcja `copy_selected_files`
Funkcja odpowiadająca za przeprowadzenie kopiowania wybranych plików

In [4]:
def copy_selected_files(src_dir, dst_dir, selected_ids, file_index,):
    dst_dir.mkdir(parents=True, exist_ok=True)
    for stem in selected_ids:
        src = file_index.get(stem)
        
        if src and src.is_file():
            shutil.copy2(src, dst_dir / src.name)
        else:
            print(F"Brak pliku: {stem}")
    print(f"Skopiowano {len(selected_ids)} plików z `{src_dir.name}` do `{dst_dir}`")

#### Funkcja `verify_consistency`
Funkcja odpowiadająca za przeprowadzenie weryfikacji czy nowo powstałe foldery są spójne względem siebie

In [5]:
def verify_consistency(dir_a, dir_b):
    a = {p.stem for p in dir_a.iterdir() if p.is_file()}
    b = {p.stem for p in dir_b.iterdir() if p.is_file()}
    only_a = a - b
    only_b = b - a

    if not only_a and not only_b:
        print("Foldery są spójne.")
    else:
        if only_a:
            print(f"Tylko w `{dir_a.name}`:", only_a)
        if only_b:
            print(f"Tylko w `{dir_b.name}`:", only_b)
    print(f"Podsumowanie: `{dir_a.name}`={len(a)}, `{dir_b.name}`={len(b)}")

#### Funkcja `process_copy`
Funkcja grupująca wywołanie wszystkich funkcji potrzebnych do poprawnego przeprowadzenia kopiowania plików

In [6]:
def process_copy(images_dir, masks_dir, csv_path, output_base, split_name = "", id_field = "image_id"):
    good_ids = load_ids(csv_path, id_field)

    idx_imgs  = build_file_index(images_dir)
    idx_masks = build_file_index(masks_dir)

    if split_name:
        out_imgs  = output_base / split_name / "images"
        out_masks = output_base / split_name / "masks"
    else:
        out_imgs  = output_base / "images"
        out_masks = output_base / "masks"

    copy_selected_files(images_dir, out_imgs, good_ids, idx_imgs)
    copy_selected_files(masks_dir, out_masks, good_ids, idx_masks)
    verify_consistency(out_imgs, out_masks)